In [600]:
from project_lib import Project
project = Project()
from ibm_watson_studio_lib import access_project_or_space
wslib = access_project_or_space()

In [602]:
from json import loads
from IPython.display import JSON

In [603]:
%env USERNAME=admin
%env PASSWORD=uoKsHWi3obej
%env DATAPLATFORM_URL=https://cpd-wml.apps.neptune.cp.fyre.ibm.com

%env SPACE_ID=7b648ffc-53e3-4b16-8e59-3c55ef755f31
#db947328-7845-4f4b-a87c-2ca721960d74



env: USERNAME=admin
env: PASSWORD=uoKsHWi3obej
env: DATAPLATFORM_URL=https://cpd-wml.apps.neptune.cp.fyre.ibm.com
env: SPACE_ID=7b648ffc-53e3-4b16-8e59-3c55ef755f31


In [604]:
%env SPACE_ID  
#new space

'7b648ffc-53e3-4b16-8e59-3c55ef755f31'

In [605]:
wml_credentials = {
    "username": "admin",
    "passowrd": "uoKsHWi3obej",
    "url": "https://cpd-wml.apps.neptune.cp.fyre.ibm.com",
    "instance_id": 'openshift',
    "version": '4.7'
}

In [606]:
from ibm_watson_machine_learning import APIClient

client = APIClient(wml_credentials)

client.set.default_space("1ad236d9-2b4d-4853-826f-358f58212ad2")
#db947328-7845-4f4b-a87c-2ca721960d74")

'SUCCESS'

#### get auth token

In [607]:
%%bash --out token

token=$(curl -sk -X POST \
        --header "Content-type: application/json" \
        -d "{\"username\":\"${USERNAME}\",\"password\":\"${PASSWORD}\"}" \
        "$DATAPLATFORM_URL/icp4d-api/v1/authorize")

token=${token#*token\":\"}
token=${token%%\"*}
echo $token

In [608]:
%env TOKEN=$token

env: TOKEN=eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCIsImtpZCI6ImVZa2h5dGxMelFBcGt4ZUFxR3NIOGVaYTA3aXhqMXlzcVJzYTJLdnlHdncifQ.eyJ1c2VybmFtZSI6ImFkbWluIiwicm9sZSI6IkFkbWluIiwicGVybWlzc2lvbnMiOlsiY3JlYXRlX3Byb2plY3QiLCJtYW5hZ2VfY2F0YWxvZyIsIm1hbmFnZV92YXVsdHNfYW5kX3NlY3JldHMiLCJzaGFyZV9zZWNyZXRzIiwiYWRkX3ZhdWx0cyIsImFkbWluaXN0cmF0b3IiLCJjYW5fcHJvdmlzaW9uIiwibW9uaXRvcl9wbGF0Zm9ybSIsImNvbmZpZ3VyZV9wbGF0Zm9ybSIsInZpZXdfcGxhdGZvcm1faGVhbHRoIiwiY29uZmlndXJlX2F1dGgiLCJtYW5hZ2VfdXNlcnMiLCJtYW5hZ2VfZ3JvdXBzIiwibWFuYWdlX3NlcnZpY2VfaW5zdGFuY2VzIiwiY3JlYXRlX3NwYWNlIiwiYWNjZXNzX2NhdGFsb2ciXSwiZ3JvdXBzIjpbMTAwMDBdLCJzdWIiOiJhZG1pbiIsImlzcyI6IktOT1hTU08iLCJhdWQiOiJEU1giLCJ1aWQiOiIxMDAwMzMwOTk5IiwiYXV0aGVudGljYXRvciI6ImRlZmF1bHQiLCJkaXNwbGF5X25hbWUiOiJhZG1pbiIsImFwaV9yZXF1ZXN0IjpmYWxzZSwiaWF0IjoxNjk4NzA4NjI5LCJleHAiOjE2OTg3NTE4Mjl9.EMdRcRgImoWstrM11gpyFIZYw6TtC669-wWZzIKd3ThYlGFFwxAhLDlvo60xMnXTOLZmqZsF7ct9Q2Uvh5v0ba2ZWtboKImUPOYncXsWi6kl8t3SfGorgCpLqAzDh0eMc6JpztfcV8XeiW-CkIIJe00CtVxxV4Sb9tCjFR55fVm4ji1y2XYxQ

#### store model

In [337]:
%%bash --out model_payload

MODEL_PAYLOAD='{"space_id": "'"$SPACE_ID"'","name": "scikit_diabetes_model","description": "","type": "scikit-learn_1.1", "software_spec": {"name": "runtime-22.2-py3.10"}}'
echo $MODEL_PAYLOAD

In [338]:
JSON(loads(model_payload))

<IPython.core.display.JSON object>

In [339]:
%env MODEL_PAYLOAD=$model_payload

env: MODEL_PAYLOAD={"space_id": "7b648ffc-53e3-4b16-8e59-3c55ef755f31","name": "scikit_diabetes_model","description": "","type": "scikit-learn_1.1", "software_spec": {"name": "runtime-22.2-py3.10"}}


In [340]:
%%bash --out model_id -s "$model_payload"

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$MODEL_PAYLOAD" \
    "$DATAPLATFORM_URL/ml/v4/models?version=2023-10-25" | grep '"id": ' | awk -F '"' '{ print $4 }' | sed -n 2p

In [341]:
%env MODEL_ID=$model_id

env: MODEL_ID=d535821f-a9b5-4772-b654-7f03114efd6c


#### store an external model

In [334]:
%%bash --out model_payload1

MODEL_PAYLOAD1='{"space_id": "'"$SPACE_ID"'","name": "scikit_diabetes_model","description": "This is description","type": "scikit-learn_1.1", "software_spec": {"name": "runtime-22.1-py3.10"}}'
echo $MODEL_PAYLOAD1


In [ ]:
%env MODEL_PAYLOAD1=$model_payload1

In [335]:
%%bash --out model_id1 -s "$MODEL_PAYLOAD1"

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$MODEL_PAYLOAD1" \
    "$DATAPLATFORM_URL/ml/v4/models?version=2023-10-24" | grep '"id": ' | awk -F '"' '{ print $4 }' | sed -n 2p

In [336]:
%env MODEL_ID=$model_id1

env: MODEL_ID=92157bc8-a81f-4109-9a59-5c1f3e5a4434


#### create a revision

In [342]:
%%bash --out revision_payload

REVISION_PAYLOAD='{"space_id": "'"$SPACE_ID"'", "commit_message": "Initial model."}'
echo $REVISION_PAYLOAD

In [343]:
%env REVISION_PAYLOAD=$revision_payload

env: REVISION_PAYLOAD={"space_id": "7b648ffc-53e3-4b16-8e59-3c55ef755f31", "commit_message": "Initial model."}


In [344]:
%%bash

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$REVISION_PAYLOAD" \
    "$DATAPLATFORM_URL/ml/v4/models/$MODEL_ID/revisions?version=2023-10-25" \
    | python -m json.tool

{
    "entity": {
        "hybrid_pipeline_software_specs": [],
        "software_spec": {
            "id": "b56101f1-309d-549b-a849-eaa63f77b2fb",
            "name": "runtime-22.2-py3.10"
        },
        "type": "scikit-learn_1.1"
    },
    "metadata": {
        "commit_info": {
            "commit_message": "Initial model.",
            "committed_at": "2023-10-30T17:33:44.002Z"
        },
        "created_at": "2023-10-30T17:32:23.353Z",
        "id": "d535821f-a9b5-4772-b654-7f03114efd6c",
        "modified_at": "2023-10-30T17:32:23.353Z",
        "name": "scikit_diabetes_model",
        "owner": "1000330999",
        "resource_key": "db84066c-aa84-4213-b106-18a38fbe6809",
        "rev": "1",
        "space_id": "7b648ffc-53e3-4b16-8e59-3c55ef755f31"
    },
    "system": {
        "warnings": []
    }
}


#### upload model content

In [346]:
!wget -q https://github.com/IBM/watson-machine-learning-samples/raw/master/cpd4.0/models/scikit/diabetes/model/diabetes_model.tar.gz \
    -O diabetes_model.tar.gz 

In [348]:
%%bash --out attachment_id

curl -sk -X PUT \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/gzip" \
    --header "Accept: application/json" \
    --data-binary "@diabetes_model.tar.gz" \
    "$DATAPLATFORM_URL/ml/v4/models/$MODEL_ID/content?space_id=$SPACE_ID&version=2020-08-01&content_format=native" \
    | grep "attachment_id" | awk -F '"' '{ print $4 }'


In [349]:
%env ATTACHMENT_ID=$attachment_id

env: ATTACHMENT_ID=85d85f4b-5776-4d26-93dc-c8e2875459b3


#### download a model

In [350]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --output "model.tar.gz" \
    "$DATAPLATFORM_URL/ml/v4/models/$MODEL_ID/download?space_id=$SPACE_ID&version=2020-08-01"

In [351]:
!ls -l model.tar.gz

-rw-rw----. 1 1000690000 wscommon 5322360 Oct 30 17:39 model.tar.gz


#### create deployment

In [352]:
%%bash --out deployment_payload

DEPLOYMENT_PAYLOAD='{"space_id": "'"$SPACE_ID"'","name": "Diabetes Model deployment", "description": "This is description","online": {},
"hardware_spec": {"name": "S"},"asset": {"id": "'"$MODEL_ID"'"}}'
echo $DEPLOYMENT_PAYLOAD

In [353]:
JSON(loads(deployment_payload))


<IPython.core.display.JSON object>

In [354]:
%env DEPLOYMENT_PAYLOAD=$deployment_payload

env: DEPLOYMENT_PAYLOAD={"space_id": "7b648ffc-53e3-4b16-8e59-3c55ef755f31","name": "Diabetes Model deployment", "description": "This is description","online": {},"hardware_spec": {"name": "S"},"asset": {"id": "d535821f-a9b5-4772-b654-7f03114efd6c"}}


In [355]:
%%bash --out deployment_id

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$DEPLOYMENT_PAYLOAD" \
    "$DATAPLATFORM_URL/ml/v4/deployments?version=2020-08-01" | grep '"id": ' | awk -F '"' '{ print $4 }' | sed -n 3p
    


In [356]:
%env DEPLOYMENT_ID=$deployment_id

env: DEPLOYMENT_ID=40458de1-f510-4fe4-856c-e29028a33de4


#### get deployment details

In [357]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    "$DATAPLATFORM_URL/ml/v4/deployments/$DEPLOYMENT_ID?space_id=$SPACE_ID&version=2025-10-25" \
    | python -m json.tool


{
    "entity": {
        "asset": {
            "id": "d535821f-a9b5-4772-b654-7f03114efd6c"
        },
        "custom": {},
        "deployed_asset_type": "model",
        "description": "This is description",
        "hardware_spec": {
            "id": "e7ed1d6c-2e89-42d7-aed5-863b972c1d2b",
            "name": "S",
            "num_nodes": 1
        },
        "name": "Diabetes Model deployment",
        "online": {},
        "space_id": "7b648ffc-53e3-4b16-8e59-3c55ef755f31",
        "status": {
            "online_url": {
                "url": "https://cpd-wml.apps.neptune.cp.fyre.ibm.com/ml/v4/deployments/40458de1-f510-4fe4-856c-e29028a33de4/predictions"
            },
            "serving_urls": [
                "https://cpd-wml.apps.neptune.cp.fyre.ibm.com/ml/v4/deployments/40458de1-f510-4fe4-856c-e29028a33de4/predictions"
            ],
            "state": "ready"
        }
    },
    "metadata": {
        "created_at": "2023-10-30T17:39:59.586Z",
        "description": 

#### score

In [358]:
%%bash --out scoring_payload

SCORING_PAYLOAD='{"space_id": "$SPACE_ID","input_data": [{"fields": ["age", "sex", "bmi", "bp", "s1", "s2", "s3", "s4", "s5", "s6"], "values": [[-0.00188201652779104, -0.044641636506989, -0.0514740612388061, -0.0263278347173518, -0.00844872411121698, -0.019163339748222, 0.0744115640787594, -0.0394933828740919, -0.0683297436244215, -0.09220404962683], [0.0852989062966783, 0.0506801187398187, 0.0444512133365941, -0.00567061055493425, -0.0455994512826475, -0.0341944659141195, -0.0323559322397657, -0.00259226199818282, 0.00286377051894013, -0.0259303389894746]]}]}'
echo $SCORING_PAYLOAD

In [359]:
JSON(loads(scoring_payload))

<IPython.core.display.JSON object>

In [360]:
%env SCORING_PAYLOAD=$scoring_payload

env: SCORING_PAYLOAD={"space_id": "$SPACE_ID","input_data": [{"fields": ["age", "sex", "bmi", "bp", "s1", "s2", "s3", "s4", "s5", "s6"], "values": [[-0.00188201652779104, -0.044641636506989, -0.0514740612388061, -0.0263278347173518, -0.00844872411121698, -0.019163339748222, 0.0744115640787594, -0.0394933828740919, -0.0683297436244215, -0.09220404962683], [0.0852989062966783, 0.0506801187398187, 0.0444512133365941, -0.00567061055493425, -0.0455994512826475, -0.0341944659141195, -0.0323559322397657, -0.00259226199818282, 0.00286377051894013, -0.0259303389894746]]}]}


In [361]:
%%bash

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$SCORING_PAYLOAD"\
    "$DATAPLATFORM_URL/ml/v4/deployments/$DEPLOYMENT_ID/predictions?version=2023-10-25" \
    | python -m json.tool

{
    "predictions": [
        {
            "fields": [
                "prediction"
            ],
            "values": [
                [
                    82.3468335692293
                ],
                [
                    168.2544647515973
                ]
            ]
        }
    ]
}


#### list all deployments

In [362]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    "$DATAPLATFORM_URL/ml/v4/deployments?space_id=$SPACE_ID&version=2023-10-25" \
    | python -m json.tool

{
    "resources": [
        {
            "entity": {
                "asset": {
                    "id": "d535821f-a9b5-4772-b654-7f03114efd6c"
                },
                "custom": {},
                "deployed_asset_type": "model",
                "description": "This is description",
                "hardware_spec": {
                    "id": "e7ed1d6c-2e89-42d7-aed5-863b972c1d2b",
                    "name": "S",
                    "num_nodes": 1
                },
                "name": "Diabetes Model deployment",
                "online": {},
                "space_id": "7b648ffc-53e3-4b16-8e59-3c55ef755f31",
                "status": {
                    "online_url": {
                        "url": "https://cpd-wml.apps.neptune.cp.fyre.ibm.com/ml/v4/deployments/40458de1-f510-4fe4-856c-e29028a33de4/predictions"
                    },
                    "serving_urls": [
                        "https://cpd-wml.apps.neptune.cp.fyre.ibm.com/ml/v4/deployments/40458

#### model update

#### list the revisions

In [363]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    "$DATAPLATFORM_URL/ml/v4/models/$MODEL_ID/revisions?space_id=$SPACE_ID&version=2023-10-25" \
    | python -m json.tool

{
    "first": {
        "href": "/ml/v4/models/d535821f-a9b5-4772-b654-7f03114efd6c/revisions?space_id=7b648ffc-53e3-4b16-8e59-3c55ef755f31&limit=100"
    },
    "limit": 100,
    "resources": [
        {
            "metadata": {
                "commit_info": {
                    "commit_message": "Initial model.",
                    "committed_at": "2023-10-30T17:33:44.002Z"
                },
                "created_at": "2023-10-30T17:32:23.353Z",
                "id": "d535821f-a9b5-4772-b654-7f03114efd6c",
                "modified_at": "2023-10-30T17:32:23.353Z",
                "name": "scikit_diabetes_model",
                "owner": "1000330999",
                "resource_key": "db84066c-aa84-4213-b106-18a38fbe6809",
                "rev": "1",
                "space_id": "7b648ffc-53e3-4b16-8e59-3c55ef755f31"
            }
        }
    ]
}


#### create second revision

In [364]:
%%bash --out revision_payload

REVISION_PAYLOAD='{"space_id": "'"$SPACE_ID"'", "commit_message": "Updated model."}'
echo $REVISION_PAYLOAD

In [366]:
JSON(loads(revision_payload))


<IPython.core.display.JSON object>

In [367]:
%env REVISION_PAYLOAD=$revision_payload

env: REVISION_PAYLOAD={"space_id": "7b648ffc-53e3-4b16-8e59-3c55ef755f31", "commit_message": "Updated model."}


In [368]:
%%bash

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$REVISION_PAYLOAD" \
    "$DATAPLATFORM_URL/ml/v4/models/$MODEL_ID/revisions?version=2023-10-25" \
    | python -m json.tool

{
    "entity": {
        "hybrid_pipeline_software_specs": [],
        "software_spec": {
            "id": "b56101f1-309d-549b-a849-eaa63f77b2fb",
            "name": "runtime-22.2-py3.10"
        },
        "type": "scikit-learn_1.1"
    },
    "metadata": {
        "commit_info": {
            "commit_message": "Updated model.",
            "committed_at": "2023-10-30T17:41:02.002Z"
        },
        "created_at": "2023-10-30T17:32:23.353Z",
        "id": "d535821f-a9b5-4772-b654-7f03114efd6c",
        "modified_at": "2023-10-30T17:38:52.745Z",
        "name": "scikit_diabetes_model",
        "owner": "1000330999",
        "resource_key": "7600a9b5-e498-4fad-84af-9d781cda194b",
        "rev": "2",
        "space_id": "7b648ffc-53e3-4b16-8e59-3c55ef755f31"
    },
    "system": {
        "warnings": []
    }
}


#### update model metadata

In [369]:
%%bash --out update_payload

echo '[{"op": "add", "path": "/name", "value": "updated scikit model"}]'

In [370]:
%env UPDATE_PAYLOAD=$update_payload

env: UPDATE_PAYLOAD=[{"op": "add", "path": "/name", "value": "updated scikit model"}]


In [371]:
%%bash

curl -sk -X PATCH \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$UPDATE_PAYLOAD" \
    "$DATAPLATFORM_URL/ml/v4/models/$MODEL_ID?space_id=$SPACE_ID&version=2023-10-25" \
    | python -m json.tool


{
    "entity": {
        "hybrid_pipeline_software_specs": [],
        "software_spec": {
            "id": "b56101f1-309d-549b-a849-eaa63f77b2fb",
            "name": "runtime-22.2-py3.10"
        },
        "type": "scikit-learn_1.1"
    },
    "metadata": {
        "commit_info": {
            "committed_at": "2023-10-30T17:41:07.994Z"
        },
        "created_at": "2023-10-30T17:32:23.353Z",
        "id": "d535821f-a9b5-4772-b654-7f03114efd6c",
        "modified_at": "2023-10-30T17:41:07.948Z",
        "name": "updated scikit model",
        "owner": "1000330999",
        "resource_key": "7600a9b5-e498-4fad-84af-9d781cda194b",
        "space_id": "7b648ffc-53e3-4b16-8e59-3c55ef755f31"
    },
    "system": {
        "warnings": []
    }
}


#### upload the new model content

In [372]:
!wget -q https://github.com/IBM/watson-machine-learning-samples/raw/master/cpd4.0/models/scikit/diabetes/model/new_diabetes_model.tar.gz \
    -O new_diabetes_model.tar.gz 

In [373]:
%%bash

curl -sk -X PUT \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/gzip" \
    --header "Accept: application/json" \
    --data-binary "@new_diabetes_model.tar.gz" \
    "$DATAPLATFORM_URL/ml/v4/models/$MODEL_ID/content?space_id=$SPACE_ID&version=2023-10-25&content_format=native" \
    | python -m json.tool

{
    "attachment_id": "c7b7bdb9-d30f-4ef0-a636-bbb4f0a3c2e1",
    "content_format": "native",
    "persisted": true
}


#### get model revisions 

In [374]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    "$DATAPLATFORM_URL/ml/v4/models/$MODEL_ID/revisions?space_id=$SPACE_ID&version=2023-10-25" \
    | python -m json.tool

{
    "first": {
        "href": "/ml/v4/models/d535821f-a9b5-4772-b654-7f03114efd6c/revisions?space_id=7b648ffc-53e3-4b16-8e59-3c55ef755f31&limit=100"
    },
    "limit": 100,
    "resources": [
        {
            "metadata": {
                "commit_info": {
                    "commit_message": "Updated model.",
                    "committed_at": "2023-10-30T17:41:02.002Z"
                },
                "created_at": "2023-10-30T17:32:23.353Z",
                "id": "d535821f-a9b5-4772-b654-7f03114efd6c",
                "modified_at": "2023-10-30T17:38:52.745Z",
                "name": "scikit_diabetes_model",
                "owner": "1000330999",
                "resource_key": "7600a9b5-e498-4fad-84af-9d781cda194b",
                "rev": "2",
                "space_id": "7b648ffc-53e3-4b16-8e59-3c55ef755f31"
            }
        },
        {
            "metadata": {
                "commit_info": {
                    "commit_message": "Initial model.",
             

In [90]:
#### get model details

In [375]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    "$DATAPLATFORM_URL/ml/v4/models/$MODEL_ID?space_id=$SPACE_ID&version=2023-10-25" \
    | python -m json.tool

{
    "entity": {
        "hybrid_pipeline_software_specs": [],
        "software_spec": {
            "id": "b56101f1-309d-549b-a849-eaa63f77b2fb",
            "name": "runtime-22.2-py3.10"
        },
        "type": "scikit-learn_1.1"
    },
    "metadata": {
        "commit_info": {
            "committed_at": "2023-10-30T17:41:24.734Z"
        },
        "created_at": "2023-10-30T17:32:23.353Z",
        "id": "d535821f-a9b5-4772-b654-7f03114efd6c",
        "modified_at": "2023-10-30T17:41:17.582Z",
        "name": "updated scikit model",
        "owner": "1000330999",
        "resource_key": "ca9dc29d-52f7-4025-b560-aac25fd7a487",
        "space_id": "7b648ffc-53e3-4b16-8e59-3c55ef755f31"
    },
    "system": {
        "warnings": []
    }
}


#### redeploy and score the new version

In [376]:
%%bash --out redeploy_payload

echo '[{"op": "replace", "path": "/asset", "value": {"id": "'"$MODEL_ID"'"}}]'

In [377]:
%env REDEPLOY_PAYLOAD=$redeploy_payload

env: REDEPLOY_PAYLOAD=[{"op": "replace", "path": "/asset", "value": {"id": "d535821f-a9b5-4772-b654-7f03114efd6c"}}]


In [378]:
%%bash

curl -sk -X PATCH \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$REDEPLOY_PAYLOAD" \
    "$DATAPLATFORM_URL/ml/v4/deployments/$DEPLOYMENT_ID?space_id=$SPACE_ID&version=2023-10-25" \
    | python -m json.tool

{
    "entity": {
        "asset": {
            "id": "d535821f-a9b5-4772-b654-7f03114efd6c"
        },
        "custom": {},
        "deployed_asset_type": "model",
        "description": "This is description",
        "hardware_spec": {
            "id": "e7ed1d6c-2e89-42d7-aed5-863b972c1d2b",
            "name": "S",
            "num_nodes": 1
        },
        "name": "Diabetes Model deployment",
        "online": {},
        "space_id": "7b648ffc-53e3-4b16-8e59-3c55ef755f31",
        "status": {
            "online_url": {
                "url": "https://cpd-wml.apps.neptune.cp.fyre.ibm.com/ml/v4/deployments/40458de1-f510-4fe4-856c-e29028a33de4/predictions"
            },
            "serving_urls": [
                "https://cpd-wml.apps.neptune.cp.fyre.ibm.com/ml/v4/deployments/40458de1-f510-4fe4-856c-e29028a33de4/predictions"
            ],
            "state": "ready"
        }
    },
    "metadata": {
        "created_at": "2023-10-30T17:39:59.586Z",
        "description": 

In [379]:
%%bash

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$SCORING_PAYLOAD" \
    "$DATAPLATFORM_URL/ml/v4/deployments/$DEPLOYMENT_ID/predictions?version=2023-10-25" \
    | python -m json.tool


{
    "predictions": [
        {
            "fields": [
                "prediction"
            ],
            "values": [
                [
                    82.3468335692293
                ],
                [
                    168.2544647515973
                ]
            ]
        }
    ]
}
